In [ ]:
import os
pathtovideos = '/home/mike/Documents/Research/bee_identification_project/videos'
import requests
import json
import time
#for vid,data in df.iterrows():
#    print(vid)
#    os.system('ffmpeg -i %s -r 0.25 %s-image-%%3d.jpeg' % (pathtovideos+'/'+vid,'imagesfromvideos/'+vid[:-4]))        


### THEN NEED TO RUN MARCIN'S LOCALISER TOOL...
#e.g. download yolov5_scripts
python3 detect_scripts.py --source /home/mike/Documents/Research/SheffieldMLpublications/publications/airvi/jupyter/label_cal/imagesfromvideos --weights models/bumblebee_weigths.pt --conf 0.4 --crop

### THEN CALL JENNIFER'S CNN...
from glob import glob
allscores = []
labels = []
for vid,data in df.iterrows():
    print(vid)
    scores = {}
    for fn in glob('/home/mike/Documents/Research/yolov5_scripts/inference/output/cropped/'+vid[:-4]+'*_CROPPED.jpg'):
        try:
            response = requests.post('http://52.16.188.225/api/image', files={'image1': ('bee.jpeg', open(fn,'rb'))})
            for sp,p in json.loads(response.text)['pred']:
                if sp not in scores: scores[sp]=[]
                scores[sp].append(p)
        except FileNotFoundError:
            break
        time.sleep(1)
    avgscores = {}
    print("scores")
    print(scores)
    for sp, l in scores.items():
        avgscores[sp] = np.prod(l) #product of probabilities
        
    wb = 0
    if 'Bombus lucorum' in avgscores: 
        wb += avgscores['Bombus lucorum']
        avgscores.pop('Bombus lucorum')
    if 'Bombus terrestris' in avgscores: 
        wb += avgscores['Bombus terrestris']
        avgscores.pop('Bombus terrestris')
    
    avgscores['whitebuff'] = wb
    a = np.array(list(avgscores.items()))
    labels.append(a[np.argsort(-a[:,1].astype(float))][0][0])
    print("labels")
    print(labels)
    allscores.append(scores)
    #print("allscores")
    #print(allscores)